## Libraries and Loading

In [78]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
from collections import Counter

In [79]:
Complete_data = r"C:\Frank\UoT 2024-2025\MIE8888 Project\Data\Complete Data\21_treatements_backup_joined_patients_procedure_na_removed.csv"
Patient_Procedure = r"C:\Frank\UoT 2024-2025\MIE8888 Project\Data\Complete Data\21_treatements_backup_joined_patients_procedure_grouped.csv"

## Encoding Type Helper Function

In [80]:
# with open(Complete_data, 'rb') as f:
#     raw_data = f.read()
#     result = chardet.detect(raw_data)
#     encoding = result['encoding']
# print(f"Detected encoding: {encoding}")

In [81]:
# with open(Patient_Procedure, 'rb') as f:
#     raw_data = f.read()
#     result = chardet.detect(raw_data)
#     encoding2 = result['encoding']
# print(f"Detected encoding: {encoding}")

## Temporarily remove empty columns

In [82]:
df = pd.read_csv(Complete_data, encoding="Windows-1252")
cleaned_df = df.dropna(axis=1, how='all')
cleaned_column_names = cleaned_df.columns.tolist()
print(cleaned_column_names)
print(cleaned_df.shape[1])

original_column_names = df.columns.tolist()
dropped_columns = [col for col in original_column_names if col not in cleaned_column_names]
print(dropped_columns)
print(len(dropped_columns))

C:\Users\frank\AppData\Local\Temp\ipykernel_13928\2635263723.py:1: DtypeWarning: Columns (1,4,10,16,52,85,86,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Complete_data, encoding="Windows-1252")


['id_x_x', 'sikka_sub_task_x_x', 'sikka_sub_task_operation_x_x', 'name_x', 'office_x_x', 'patient', 'guarantor_x', 'provider_x', 'description', 'state_x', 'visible_to_patient', 'amount', 'estimated_cost', 'insurance_coverage', 'tooth_from', 'primary_insurance_estimate', 'secondary_insurance_estimate', 'plan_sr_no', 'entry_date', 'insurance_payment', 'treatment_plan_status', 'procedure_date', 'priority', 'approved', 'from_pms_x_x', 'id_y_x', 'from_pms_y_x', 'sikka_sub_task_y_x', 'sikka_sub_task_operation_y_x', 'uuid', 'patient_id', 'guarantor_y', 'office_y_x', 'fullname', 'first_name', 'middle_name', 'last_name', 'birth_date', 'address', 'city', 'state_y', 'first_visit', 'last_visit', 'email', 'phone', 'zip_code', 'practice', 'sikka_provider_id', 'primary_insurance_company_id', 'profile_picture', 'date_joined', 'zip_code_flag', 'address_comp', 'fetched_zip_code', 'fetched_zip_code_flag', 'zip_code_verified', 'final_zip_code', 'LATITUDE', 'LONGITUDE', 'id_x_y', 'from_pms_x_y', 'sikka_sub

## Task 1: Focus on the treatment processing per day, consider the ones with more than 7 treatment per day.

In [92]:
df2 = pd.read_csv(Patient_Procedure)
filtered_df = df2[df2['procedure_code_y_len'] >= 7]
unnormal_case = filtered_df[['procedure_code_y', 'procedure_code_y_len']]
unnormal_case.head()

,procedure_code_y,procedure_code_y_len
939,"[1203, 2142, 12101, 43412, 62501, 67211, 99111]",7
1458,"[1103, 1103, 2113, 2113, 2144, 2144, 2601, 260...",10
1475,"[1103, 2142, 2601, 11112, 11117, 12101, 99600]",7
2810,"[1103, 2112, 2142, 2601, 12101, 43412, 43413]",7
3156,"[1202, 2114, 11101, 11113, 12101, 25531, 27602...",9


In [94]:
case_counts = Counter(unnormal_case['procedure_code_y'])
case_summary = pd.DataFrame.from_dict(case_counts, orient='index', columns=['Count'])
case_summary.head()

,Count
"[1203, 2142, 12101, 43412, 62501, 67211, 99111]",1
"[1103, 1103, 2113, 2113, 2144, 2144, 2601, 2601, 23311, 23311]",1
"[1103, 2142, 2601, 11112, 11117, 12101, 99600]",1
"[1103, 2112, 2142, 2601, 12101, 43412, 43413]",1
"[1202, 2114, 11101, 11113, 12101, 25531, 27602, 99111, 99111]",1


In [95]:
# Consider the count of less than 10 as ramdom occurrence
case_summary = case_summary[case_summary['Count'] >= 10]
case_summary.head()

,Count
"[1202, 2144, 2144, 11107, 11107, 11113, 11113]",23
"[1103, 2112, 2144, 2601, 11103, 12101, 99700]",41
"[1202, 1202, 11107, 11107, 11113, 11113, 12111, 12111]",11
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113]",29
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113, 12111, 12111]",11


In [96]:
# Get the procedure code for special case and use them to find the treatment description in full dataset
unique_numbers = set()
for idx in case_summary.index:
    numbers = eval(idx) if isinstance(idx, str) else idx
    unique_numbers.update(numbers)
unique_numbers = sorted(unique_numbers)
print("Unique Codes", unique_numbers)


Unique Codes [1103, 1202, 2112, 2142, 2144, 2601, 11101, 11103, 11107, 11111, 11113, 11117, 12101, 12111, 96103, 99700]


In [97]:
special_descriptions = []
for i in unique_numbers:
    matching_rows = df[df['procedure_code_y'] == i]
    if not matching_rows.empty:
        special_descriptions.append({
            'procedure_code_y': i,
            'special_code_description': matching_rows['procedure_code_description'].iloc[0],  # Take the first match
            'count': len(matching_rows)
        })
summary = pd.DataFrame(special_descriptions).sort_values('count', ascending=False)
summary


,procedure_code_y,special_code_description,count
12,12101,!!!Fluoride Treatment Foam Gel or Rinse,25357
10,11113,Scaling Three Units,22911
1,1202,Exam & Diagnosis Limited Oral Previous Patient...,21516
6,11101,Polishing One Unit of Time,16154
11,11117,Scaling 1/2 Unit,15616
8,11107,Polishing 1/2 Unit of Time,15010
3,2142,Radiographs Intraoral Bitewing-2 Images,14629
4,2144,Radiographs Intraoral Bitewing - 4 Images,12256
5,2601,Radiographs Panoramic - 1 Image,11676
9,11111,Scaling One Unit of Time,7780


In [98]:
code_to_description = dict(zip(summary['procedure_code_y'], summary['special_code_description']))
case_summary['Description'] = case_summary.index.map(lambda codes: ", ".join([code_to_description[int(code)] for code in eval(codes) if int(code) in code_to_description]))
case_summary.sort_values('Count', ascending=False)

,Count,Description
"[1103, 2112, 2144, 2601, 11103, 12101, 99700]",41,Exam and Diagnosis Complete - Permanent Dentit...
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113]",29,Exam & Diagnosis Limited Oral Previous Patient...
"[1202, 2144, 2144, 11107, 11107, 11113, 11113]",23,Exam & Diagnosis Limited Oral Previous Patient...
"[1103, 2112, 2142, 2601, 11103, 12101, 99700]",20,Exam and Diagnosis Complete - Permanent Dentit...
"[1103, 2112, 2144, 2601, 11111, 96103, 99700]",16,Exam and Diagnosis Complete - Permanent Dentit...
"[1202, 2142, 2601, 11101, 11111, 11117, 12101]",14,Exam & Diagnosis Limited Oral Previous Patient...
"[1202, 1202, 11107, 11107, 11113, 11113, 12111, 12111]",11,Exam & Diagnosis Limited Oral Previous Patient...
"[1202, 1202, 2144, 2144, 11107, 11107, 11113, 11113, 12111, 12111]",11,Exam & Diagnosis Limited Oral Previous Patient...


- Above is one table that shows the **unnormal treatment groups** (more than 7 treatments applied to one patient one day) that have happened more than 10 times in our data.  
- This table includes the description of what the treatment group as well as the exact times these treatment group appeared.
- These groups may reflect specific orders of the treatment group that doctors already knew;  
- But some are possibly not, and we can use these groups for the use of next-step-treatment prediction reference

---

## Task 2 Use aprior algorithm on co-occurrence detection